In [9]:
import numpy as np
import scipy.stats as stats
import scipy.special
#graphing
import matplotlib.pyplot as plt

#optimize
from scipy.optimize import minimize

#interpolate
from scipy.integrate import quad

In [10]:
def optimal_reserve(mu,sigma):
    v = 0
    obj = lambda p :  -1*(p-v)*(1-stats.logistic(loc=mu,scale=sigma).cdf(p))
    result = minimize(obj,[0])
    return result.x
    
print(optimal_reserve(9.3064,21.6697))
print(optimal_reserve(30.9627 ,8.1390))
print('-----')
print(optimal_reserve(9.8540,10.0752))
print(optimal_reserve(19.9341,8.7761))

[30.00633277]
[25.02345928]
-----
[15.70958205]
[18.78262703]


In [11]:
print(optimal_reserve(3.0370,1.9556))
print(optimal_reserve(7.4830 ,1.2305))
print('-----')
print(optimal_reserve(-0.0054,2.4029))
print(optimal_reserve(5.4083,1.4276))

[3.49939847]
[5.85411731]
-----
[3.07084525]
[4.37391668]


In [12]:
def expected_rev(p,mu = 3,sigma = 2,i=10):
   
    values_distr = stats.logistic(loc=mu, scale=sigma)
    cdf = lambda x : values_distr.cdf(x)
    pdf = lambda x: values_distr.pdf(x)
    factorial = scipy.special.factorial(i)/scipy.special.factorial(i-2)  
        
    order_cdf = lambda x : factorial*pdf(x)*cdf(x)**(i-2)*(1-cdf(x)) #np.log(i) + np.log(cdf) 
    integrand = lambda x : x*order_cdf(x)
    I = quad(integrand, p,100)
    return I

print(expected_rev(3.49939847,i=10))
print(expected_rev(5.85411731,i=10))
print('-------')
print(expected_rev(3.07084525,i=4))
print(expected_rev(4.37391668,i=4))

(6.3525992049582225, 1.3340628557357778e-09)
(4.532266038925901, 4.174257697651488e-12)
-------
(3.408123525423319, 2.41987273487843e-12)
(2.423474061778202, 4.78108273932183e-13)


In [14]:
def marginal_effect(beta0,beta1,sigma,x):
    mux = beta0+beta1*x
    return stats.norm.cdf(mux/np.sqrt(np.abs(sigma)))*beta1


beta0 = 1.1438
beta1 = 0.9917
sigma = 1.0772
for x in [-5,-2.5,0,2.5,5]:
    print(marginal_effect(beta0,beta1,sigma,x))

print('------')
beta0 = 1.0968
beta1 = 0.2431
sigma = 0.8248

for x in [-10,-5,0,5,10]:
    print(marginal_effect(beta0,beta1,sigma,x))

0.00011772330655832707
0.09827580524015286
0.8576026979223973
0.991461218772031
0.9916999979603717
------
0.017237008419991456
0.10891031215913055
0.21548760252498436
0.2417757828837015
0.2430875336569045


In [ ]:
support_x = np.arange(-15,20,.05)
me = marginal_effect(beta0,beta1,sigma, support_x )

me_95 = marginal_effect(beta0+ci[0]*1.96,beta1+ci[1]*1.96,
                        sigma+ci[2]*1.96, support_x )
me_05 = marginal_effect(beta0-ci[0]*1.96,beta1-ci[1]*1.96,
                        sigma-ci[2]*1.96, support_x )

#beta0 = 0.9793
#beta1 = 0.2559
#sigma = 1.0375
#ci = 0.047, 0.007, 0.049

beta0 = 1.0968
beta1 = 0.2431
sigma = 0.8248
#ci = 0.041, 0.006, 0.037

plt.plot(support_x, me ,label="mean",
         color='black')
plt.plot(support_x, me_95 ,label="95% confidence",
         color='black',linestyle='--')
plt.plot(support_x, me_05,
         color='black',linestyle='--')

plt.legend()

plt.ylabel("marginal effect")
plt.xlabel("xlabel")
plt.title("Tobit marginal effects")
plt.savefig('figs/me_weakover.png')

In [6]:
def avg_effect(beta0,beta1,sigma):
    integrand = lambda x: marginal_effect(beta0,beta1,sigma,x)*stats.norm(loc=5,scale=5).pdf(x)
    I = quad(integrand, -100,100)
    return I

beta0 = 1.1438
beta1 = 0.9917
sigma = 1.0772
print(avg_effect(beta0,beta1,sigma))

beta0 = 0.9793
beta1 = 0.2559
sigma = 1.0375
print(avg_effect(beta0,beta1,sigma))

(0.8784630588831224, 7.564433174818478e-10)
(0.23450329141984852, 3.1023089659508963e-13)
